In [2]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# 读取训练好的模型
model = load_model('predict_model.h5')

# 读取需要预测的数据
Import_csv_file = './merged_output.csv'
Import_df = pd.read_csv(Import_csv_file)

# 删除重复的 'Time' 列（如果存在）
if 'Time' in Import_df.columns.duplicated():
    Import_df = Import_df.loc[:, ~Import_df.columns.duplicated()]

# 选择输入列（从第二列到第六列，控制和动力相关特征）
X = Import_df[['AntriebsmomentVA', 'AntriebsmomentHA', 'Lenkwinkel', 'BremsmomentVA', 'BremsmomentHA']].values

# 选择输出列（如车辆状态和行为相关特征）
y = Import_df[['KS_X', 'KS_Y', 'Gierwinkel', 'Geschw_X', 'Geschw_Y', 'Giergeschwindigkeit']].values

# 归一化输入数据
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)

# 归一化输出数据
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y)

# 将数据转换为LSTM所需的时间步格式
def create_dataset(X, time_step=1):
    X_data = []
    for i in range(len(X) - time_step):
        X_data.append(X[i:(i + time_step)])
    return np.array(X_data)

time_step = 5
X_data = create_dataset(X_scaled, time_step)

# 初始化预测结果列表，将前五个时间步的真实值直接作为预测值
y_pred = list(y_scaled[:time_step])

# 从第六个时间步开始逐步预测
for i in range(time_step, len(X_scaled)):
    input_seq = np.array(y_pred[-time_step:])  # 取最近的time_step个数据
    input_seq = input_seq.reshape(1, time_step, X.shape[1])
    pred = model.predict(input_seq, verbose=0)
    y_pred.append(pred[0])

# 转换为numpy数组
y_pred = np.array(y_pred)

# 反归一化预测结果
y_pred_rescaled = scaler_y.inverse_transform(y_pred)

# 将预测结果转换为DataFrame
pred_df = pd.DataFrame(y_pred_rescaled, columns=['KS_X', 'KS_Y', 'Gierwinkel', 'Geschw_X', 'Geschw_Y', 'Giergeschwindigkeit'])

# 将预测结果与原始数据合并
merged_data = pd.concat([Import_df.reset_index(drop=True), pred_df], axis=1)

# 保存结果到CSV文件
merged_data.to_csv('pred_result.csv', index=False)
print("预测结果已保存到 pred_result.csv")


ValueError: cannot reshape array of size 30 into shape (1,5,5)